## Call API to retrieve close approach data

In [7]:
import requests
import json

#### API call for Close approach (CAD) history

In [8]:
history_url = 'https://ssd-api.jpl.nasa.gov/cad.api?date-min=1900-01-01&date-max=2021-12-31&dist-max=0.2'

#### API call for 2022 through 2100

In [9]:
curr_future_url = 'https://ssd-api.jpl.nasa.gov/cad.api?date-min=2022-01-01&date-max=2100-01-01&dist-max=0.2'

## API request

In [10]:
def make_api_request(url, file_name):
    """
    Make Api call using python requests module
    
    args:
        url: Url of API call to Nasa site
        file_name: name of file to create for data returned by Api call
    """
    response = requests.get(url)
    str_json = json.dumps(response.json())
    
    with open(file_name, 'w') as f:
        f.write(str_json)

In [11]:
# API request for historical cad data
file_name = 'cad_history.json'
make_api_request(history_url, file_name)

In [12]:
# API requset for current year and future data
file_name = 'cad.json'
make_api_request(curr_future_url, file_name)

## Upload to AWS S3 bucket

In [13]:
import boto3
import configparser

In [14]:
config = configparser.ConfigParser()
config.read('aws_config.cfg', encoding='utf-8-sig')

['aws_config.cfg']

In [15]:
AWS_KEY_ID=config['AWS_KEYS']['AWS_ACCESS_KEY_ID']
AWS_SECRET=config['AWS_KEYS']['AWS_SECRET_ACCESS_KEY']

In [31]:
file_upload_name = 'cad.json'

In [32]:
# define bucket and key names to upload Json files into
bucket_name = 'ucb-neo-project'
aws_s3_key = 'json'+'/'+file_upload_name

In [33]:
# Generate the boto3 client for interacting with S3
s3 = boto3.client('s3', region_name='us-east-1',
                  aws_access_key_id=AWS_KEY_ID, 
                  aws_secret_access_key=AWS_SECRET)

In [34]:
# Upload cad Json file to AWS S3
s3.upload_file(Bucket=bucket_name, Filename=file_upload_name, Key=aws_s3_key)

In [35]:
# Get object metadata and print it
response = s3.head_object(Bucket=bucket_name, Key=aws_s3_key)

# Print the size of the uploaded object
print(response['ContentLength'])

8988092
